# FIT3182 Assignment Part A

- Name: Tan Chong Ern
- Student ID: 31435661
- Email: ctan0119@student.monash.edu
---

## Task 1. MongoDB Data Model

### *1. Based on the two data sets provided i.e. hotspot_historic.csv and climate_historic.csv, design a suitable data model to support the efficient querying of the two data sets in MongoDB. Justify your data model design.*

### Answer
To answer this question, we must first know how the data looks like. Hence, I have used pandas to read in the CSV files of the historic climate and hotspot data. 

In [1]:
import pandas as pd
import pymongo
from pprint import pprint
from datetime import datetime

In [2]:
climate = pd.read_csv('climate_historic.csv')
climate.head()

station      date  air_temperature_celcius  relative_humidity  \
0   948700  31/12/20                       19               56.8   
1   948700    2/1/21                       15               50.7   
2   948700    3/1/21                       16               53.6   
3   948700    4/1/21                       24               61.6   
4   948700    5/1/21                       24               62.3   

   windspeed_knots  max_wind_speed precipitation   GHI_w/m2  
0              7.9            11.1          0.00I       154  
1              9.2            13.0          0.02G       128  
2              8.1            15.0          0.00G       133  
3              7.7            14.0          0.00I       186  
4              7.0            13.0          0.00I       185

In [3]:
hotspot = pd.read_csv('hotspot_historic.csv')
hotspot.head()

latitude  longitude             datetime  confidence        date  \
0   -37.966    145.051  2021-12-27T04:16:51          78  27/12/2021   
1   -35.541    143.311  2021-12-27T00:02:15          82  27/12/2021   
2   -35.554    143.307  2021-12-27T00:02:15          67  27/12/2021   
3   -35.543    143.316  2021-12-27T00:02:14          86  27/12/2021   
4   -37.708    145.100  2021-12-25T04:29:08          80  25/12/2021   

   surface_temperature_celcius  
0                           68  
1                           63  
2                           53  
3                           67  
4                           54

Now, it has been stated that, for one climate data, there can be zero to many fire (hotspot) data based on the occurence of fire(s) on a particular day. Thus, we can infer that the data from the two collections are to be linked by the date. I have chosen rows from the collections that have the same date of 27/12/2021. For simplicity, I will only use the first row of the returned hotspot rows for building the data model. 

In [4]:
climate[climate['date'] == '27/12/21']

station      date  air_temperature_celcius  relative_humidity  \
360   948702  27/12/21                       28               58.3   

     windspeed_knots  max_wind_speed precipitation   GHI_w/m2  
360              9.3            15.9          0.00I       223

In [5]:
hotspot[hotspot['date'] == '27/12/2021']

latitude  longitude             datetime  confidence        date  \
0   -37.966    145.051  2021-12-27T04:16:51          78  27/12/2021   
1   -35.541    143.311  2021-12-27T00:02:15          82  27/12/2021   
2   -35.554    143.307  2021-12-27T00:02:15          67  27/12/2021   
3   -35.543    143.316  2021-12-27T00:02:14          86  27/12/2021   

   surface_temperature_celcius  
0                           68  
1                           63  
2                           53  
3                           67

With the sample data above, we can construct an example of how the collections could look like in MongoDB, as shown below.

**Climate Collection:**
```
{
    'station': 948702,
    'date': datetime.datetime(2021, 12, 27, 0, 0),
    'air_temperature_celcius': 28,
    'relative_humidity': 58.3,
    'windspeed_knots': 9.3,
    'max_wind_speed': 15.9,
    'precipitation': 0.00,
    'precipitation_flag': 'I',
    'GHI_w/m2': 223
}
```
**Hotspot Collection:**
```
{
    'latitude': -37.966,
	'longitude': 145.051,
    'datetime': datetime.datetime(2021, 12, 27, 4, 16, 51),
    'confidence': 78,
    'date': datetime.datetime(2021, 12, 27, 0, 0),
    'surface_temperature_celcius': 68
}
```

Hence, we can now create the data model as an embedded model to support the efficient querying of the two datasets in MongoDB, as shown below.

**Data Model:**
```
{
    '_id': ObjectId('6280f3ce83911601aafcc341'),
    'station': 948702,
    'date': datetime.datetime(2021, 12, 27, 0, 0),
    'air_temperature_celcius': 28,
    'relative_humidity': 58.3,
    'windspeed_knots': 9.3,
    'max_wind_speed': 15.9,
    'precipitation': 0.00,
    'precipitation_flag': 'I',
    'GHI_w/m2': 223,
    'hotspots': [
        {
            'latitude': -37.966,
            'longitude': 145.051,
            'datetime': datetime.datetime(2021, 12, 27, 4, 16, 51),
            'confidence': 78,
            'date': datetime.datetime(2021, 12, 27, 0, 0),
            'surface_temperature_celcius': 68
        }
    ]
}
```

### Justification:

Preliminary observations of the historic climate and hotspot data shows that there is ever only one climate entry per day, while there could be multiple hotspot entries in a single day. Thus, we can infer that there is a one-to-many relationship from climate data to hotspot data. This tells us that whether we choose the embedded or reference model, the embedded document or reference key should be stored within the climate data.  

With that said, there are a few advantages to utilizing an embedded data model. Firstly, I believe that the extra space complexity in handling the collection is worth it in exchange for faster searches since there is no need for repeatedly searching in multiple collections as in a reference model. Computational complexity is kept minimal during querying since all the relevant data can be found in a single pass of the dataset and the model would be more efficient in the sense that there are less read operations required for retrieving the data. Furthermore, join operations can be avoided, which can be very heavy and complex. 

On the other hand, there are certain downsides to choosing an embedded data model. Firstly, the climate documents can quickly grow very large in size if there are a large number of hotspot entries that relate to a single climate entry. However, this may not be that big of a problem after all since MongoDB has a memory size limit of 16MB, which can hold well above 12000 climate records, and it is quite unrealistic to expect that there would be thousands of fires breaking out across Victoria in a single day. 

In conclusion, according to the points stated above, I strongly believe that the embedded data model is very suitable for our particular application in storing climate and hotspot data, which is why I have chosen it for this project. 

---

## Task 2. Querying MongoDB using PyMongo

### *1. Write a python program that will read the data from hotspot_historic.csv and climate_historic.csv and load them to the new database (e.g. fit3182_assignment_db). The collection(s) in fit3182_assignment_db will be based on the document model you have designed in Task A1.*

In [6]:
climate = pd.read_csv('climate_historic.csv')
hotspot = pd.read_csv('hotspot_historic.csv')

client = pymongo.MongoClient()
db = client.fit_3182_assignment_db
collection = db.historic
collection.drop()

# we do not have to explicitly convert each value's datatype since pandas already does it for us when reading the CSV file
climate_dict = climate.to_dict('records')
for doc in climate_dict: # get every record from the climate data (as a dictionary) and process them into the proper data format
    doc['date'] = datetime.strptime(doc['date'], '%d/%m/%y')
    precipitation = doc.pop('precipitation ')
    doc['precipitation'] = float(precipitation[:-1])
    doc['precipitation_flag'] = precipitation[-1]
    
    # get every hotspot record whose date matches the climate's and process them into the proper data format
    hotspots = hotspot[hotspot['date'] == doc['date'].strftime('%d/%m/%Y')] 
    doc['hotspots'] = []
    for hotspot_doc in hotspots.to_dict('records'):
        hotspot_doc['date'] = datetime.strptime(hotspot_doc['date'], '%d/%m/%Y')
        hotspot_doc['datetime'] = datetime.strptime(hotspot_doc['datetime'], '%Y-%m-%dT%H:%M:%S')
        doc['hotspots'].append(hotspot_doc)
    
# load the processed data into the MongoDB collection
insertion = collection.insert_many(climate_dict)
print(insertion.acknowledged)

# for x in collection.find(): pprint(x) # uncomment this line to see the newly inserted data

True


---
### *2. Write queries to answer the following tasks on fit3182_assignment_db and corresponding collection(s). You need to write the queries as a python program using the pymongo library in Jupyter Notebook.*


*a. Find climate data on 12th December 2021.*

In [7]:
res = collection.find({'date': datetime(2021, 12, 12)}, {'_id': 0, 'hotspots': 0})

for data in res:
    pprint(data)

{'GHI_w/m2': 156,
 'air_temperature_celcius': 19,
 'date': datetime.datetime(2021, 12, 12, 0, 0),
 'max_wind_speed': 12.0,
 'precipitation': 0.0,
 'precipitation_flag': 'I',
 'relative_humidity': 55.3,
 'station': 948702,
 'windspeed_knots': 6.2}


*b. Find the latitude, longitude, surface temperature (°C), and confidence when
the surface temperature (°C) was between 65 °C and 100 °C.*

In [8]:
res = collection.aggregate([{'$unwind': '$hotspots'}, 
                            {'$match': {'hotspots.surface_temperature_celcius': {'$gte': 65, '$lte': 100}}},
                            {'$project': {'_id': 0, 'hotspots.latitude': 1, 'hotspots.longitude': 1, 
                                          'hotspots.surface_temperature_celcius': 1, 'hotspots.confidence': 1}}])

for data in res:
    pprint(data)

{'hotspots': {'confidence': 94,
              'latitude': -37.2284,
              'longitude': 147.9187,
              'surface_temperature_celcius': 73}}
{'hotspots': {'confidence': 97,
              'latitude': -37.6572,
              'longitude': 142.0703,
              'surface_temperature_celcius': 80}}
{'hotspots': {'confidence': 84,
              'latitude': -37.0193,
              'longitude': 148.1459,
              'surface_temperature_celcius': 71}}
{'hotspots': {'confidence': 100,
              'latitude': -37.4229,
              'longitude': 147.027,
              'surface_temperature_celcius': 99}}
{'hotspots': {'confidence': 80,
              'latitude': -37.0055,
              'longitude': 148.1582,
              'surface_temperature_celcius': 68}}
{'hotspots': {'confidence': 85,
              'latitude': -37.4128,
              'longitude': 147.0242,
              'surface_temperature_celcius': 98}}
{'hotspots': {'confidence': 90,
              'latitude': -34.357,
   

              'surface_temperature_celcius': 67}}
{'hotspots': {'confidence': 98,
              'latitude': -36.1345,
              'longitude': 146.351,
              'surface_temperature_celcius': 84}}
{'hotspots': {'confidence': 99,
              'latitude': -36.1586,
              'longitude': 145.6666,
              'surface_temperature_celcius': 85}}
{'hotspots': {'confidence': 96,
              'latitude': -36.5847,
              'longitude': 145.1968,
              'surface_temperature_celcius': 78}}
{'hotspots': {'confidence': 93,
              'latitude': -36.9111,
              'longitude': 142.692,
              'surface_temperature_celcius': 72}}
{'hotspots': {'confidence': 99,
              'latitude': -37.5115,
              'longitude': 143.14,
              'surface_temperature_celcius': 86}}
{'hotspots': {'confidence': 86,
              'latitude': -37.6775,
              'longitude': 148.5131,
              'surface_temperature_celcius': 78}}
{'hotspots': {'confidenc

*c. Find date, surface temperature (°C), air temperature (°C), relative humidity and
max wind speed on 15th and 16th of December 2021.*

In [9]:
res = collection.find({'date': {'$in': [datetime(2021, 12, 15), datetime(2021, 12, 16)]}},
                      {'_id': 0, 'date': 1, 'hotspots.surface_temperature_celcius': 1, 
                       'air_temperature_celcius': 1, 'relative_humidity': 1, 'max_wind_speed': 1})

for data in res:
    pprint(data)

{'air_temperature_celcius': 18,
 'date': datetime.datetime(2021, 12, 15, 0, 0),
 'hotspots': [{'surface_temperature_celcius': 42},
              {'surface_temperature_celcius': 36},
              {'surface_temperature_celcius': 38},
              {'surface_temperature_celcius': 40}],
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0}
{'air_temperature_celcius': 18,
 'date': datetime.datetime(2021, 12, 16, 0, 0),
 'hotspots': [{'surface_temperature_celcius': 43},
              {'surface_temperature_celcius': 33},
              {'surface_temperature_celcius': 54},
              {'surface_temperature_celcius': 73},
              {'surface_temperature_celcius': 55},
              {'surface_temperature_celcius': 75},
              {'surface_temperature_celcius': 55},
              {'surface_temperature_celcius': 66},
              {'surface_temperature_celcius': 56},
              {'surface_temperature_celcius': 60},
              {'surface_temperature_celcius': 73},
              {'surfa

*d. Find datetime, air temperature (°C), surface temperature (°C) and confidence
when the confidence is between 80 and 100.*

In [10]:
res = collection.aggregate([{'$unwind': '$hotspots'}, 
                            {'$match': {'hotspots.confidence': {'$gte': 80, '$lte': 100}}},
                            {'$project': {'_id': 0, 'hotspots.datetime': 1, 'air_temperature_celcius': 1, 
                                          'hotspots.surface_temperature_celcius': 1, 'hotspots.confidence': 1}}])

for data in res:
    pprint(data)

{'air_temperature_celcius': 19,
 'hotspots': {'confidence': 100,
              'datetime': datetime.datetime(2021, 3, 10, 4, 48, 40),
              'surface_temperature_celcius': 105}}
{'air_temperature_celcius': 19,
 'hotspots': {'confidence': 100,
              'datetime': datetime.datetime(2021, 3, 10, 4, 46, 20),
              'surface_temperature_celcius': 109}}
{'air_temperature_celcius': 19,
 'hotspots': {'confidence': 94,
              'datetime': datetime.datetime(2021, 3, 10, 4, 45, 30),
              'surface_temperature_celcius': 73}}
{'air_temperature_celcius': 19,
 'hotspots': {'confidence': 97,
              'datetime': datetime.datetime(2021, 3, 10, 4, 45, 30),
              'surface_temperature_celcius': 80}}
{'air_temperature_celcius': 19,
 'hotspots': {'confidence': 81,
              'datetime': datetime.datetime(2021, 3, 10, 4, 43, 50),
              'surface_temperature_celcius': 55}}
{'air_temperature_celcius': 21,
 'hotspots': {'confidence': 94,
              'da

 'hotspots': {'confidence': 87,
              'datetime': datetime.datetime(2021, 3, 28, 4, 28, 30),
              'surface_temperature_celcius': 62}}
{'air_temperature_celcius': 18,
 'hotspots': {'confidence': 97,
              'datetime': datetime.datetime(2021, 3, 28, 4, 27, 40),
              'surface_temperature_celcius': 80}}
{'air_temperature_celcius': 18,
 'hotspots': {'confidence': 100,
              'datetime': datetime.datetime(2021, 3, 28, 4, 27, 20),
              'surface_temperature_celcius': 107}}
{'air_temperature_celcius': 18,
 'hotspots': {'confidence': 100,
              'datetime': datetime.datetime(2021, 3, 28, 4, 27, 10),
              'surface_temperature_celcius': 108}}
{'air_temperature_celcius': 18,
 'hotspots': {'confidence': 82,
              'datetime': datetime.datetime(2021, 3, 28, 4, 27),
              'surface_temperature_celcius': 55}}
{'air_temperature_celcius': 18,
 'hotspots': {'confidence': 99,
              'datetime': datetime.datetime(2021, 3, 

{'air_temperature_celcius': 16,
 'hotspots': {'confidence': 84,
              'datetime': datetime.datetime(2021, 4, 13, 4, 26, 50),
              'surface_temperature_celcius': 58}}
{'air_temperature_celcius': 16,
 'hotspots': {'confidence': 85,
              'datetime': datetime.datetime(2021, 4, 13, 4, 26, 50),
              'surface_temperature_celcius': 59}}
{'air_temperature_celcius': 16,
 'hotspots': {'confidence': 100,
              'datetime': datetime.datetime(2021, 4, 13, 4, 26, 40),
              'surface_temperature_celcius': 104}}
{'air_temperature_celcius': 16,
 'hotspots': {'confidence': 90,
              'datetime': datetime.datetime(2021, 4, 13, 4, 26, 40),
              'surface_temperature_celcius': 66}}
{'air_temperature_celcius': 16,
 'hotspots': {'confidence': 80,
              'datetime': datetime.datetime(2021, 4, 13, 4, 26, 40),
              'surface_temperature_celcius': 53}}
{'air_temperature_celcius': 16,
 'hotspots': {'confidence': 97,
              'date

 'hotspots': {'confidence': 84,
              'datetime': datetime.datetime(2021, 4, 13, 4, 26, 30),
              'surface_temperature_celcius': 80}}
{'air_temperature_celcius': 16,
 'hotspots': {'confidence': 80,
              'datetime': datetime.datetime(2021, 4, 13, 4, 26, 30),
              'surface_temperature_celcius': 53}}
{'air_temperature_celcius': 16,
 'hotspots': {'confidence': 91,
              'datetime': datetime.datetime(2021, 4, 13, 4, 26, 30),
              'surface_temperature_celcius': 69}}
{'air_temperature_celcius': 16,
 'hotspots': {'confidence': 86,
              'datetime': datetime.datetime(2021, 4, 13, 4, 26, 30),
              'surface_temperature_celcius': 60}}
{'air_temperature_celcius': 16,
 'hotspots': {'confidence': 100,
              'datetime': datetime.datetime(2021, 4, 13, 4, 26, 30),
              'surface_temperature_celcius': 99}}
{'air_temperature_celcius': 16,
 'hotspots': {'confidence': 88,
              'datetime': datetime.datetime(2021, 4,

{'air_temperature_celcius': 15,
 'hotspots': {'confidence': 100,
              'datetime': datetime.datetime(2021, 4, 18, 4, 44, 50),
              'surface_temperature_celcius': 114}}
{'air_temperature_celcius': 15,
 'hotspots': {'confidence': 87,
              'datetime': datetime.datetime(2021, 4, 18, 4, 44, 50),
              'surface_temperature_celcius': 62}}
{'air_temperature_celcius': 15,
 'hotspots': {'confidence': 93,
              'datetime': datetime.datetime(2021, 4, 18, 4, 44, 50),
              'surface_temperature_celcius': 72}}
{'air_temperature_celcius': 15,
 'hotspots': {'confidence': 81,
              'datetime': datetime.datetime(2021, 4, 18, 4, 44, 50),
              'surface_temperature_celcius': 54}}
{'air_temperature_celcius': 15,
 'hotspots': {'confidence': 85,
              'datetime': datetime.datetime(2021, 4, 18, 4, 44, 50),
              'surface_temperature_celcius': 60}}
{'air_temperature_celcius': 15,
 'hotspots': {'confidence': 87,
              'date

 'hotspots': {'confidence': 95,
              'datetime': datetime.datetime(2021, 4, 18, 0, 24, 50),
              'surface_temperature_celcius': 77}}
{'air_temperature_celcius': 15,
 'hotspots': {'confidence': 83,
              'datetime': datetime.datetime(2021, 4, 18, 0, 24, 40),
              'surface_temperature_celcius': 56}}
{'air_temperature_celcius': 15,
 'hotspots': {'confidence': 83,
              'datetime': datetime.datetime(2021, 4, 18, 0, 24, 40),
              'surface_temperature_celcius': 57}}
{'air_temperature_celcius': 22,
 'hotspots': {'confidence': 80,
              'datetime': datetime.datetime(2021, 4, 19, 13, 23, 10),
              'surface_temperature_celcius': 39}}
{'air_temperature_celcius': 22,
 'hotspots': {'confidence': 83,
              'datetime': datetime.datetime(2021, 4, 19, 13, 18, 20),
              'surface_temperature_celcius': 40}}
{'air_temperature_celcius': 22,
 'hotspots': {'confidence': 81,
              'datetime': datetime.datetime(2021, 4

 'hotspots': {'confidence': 87,
              'datetime': datetime.datetime(2021, 4, 29, 4, 33),
              'surface_temperature_celcius': 90}}
{'air_temperature_celcius': 10,
 'hotspots': {'confidence': 96,
              'datetime': datetime.datetime(2021, 5, 10, 4, 16, 30),
              'surface_temperature_celcius': 77}}
{'air_temperature_celcius': 10,
 'hotspots': {'confidence': 84,
              'datetime': datetime.datetime(2021, 5, 10, 4, 16, 10),
              'surface_temperature_celcius': 58}}
{'air_temperature_celcius': 10,
 'hotspots': {'confidence': 88,
              'datetime': datetime.datetime(2021, 5, 10, 4, 14, 20),
              'surface_temperature_celcius': 64}}
{'air_temperature_celcius': 10,
 'hotspots': {'confidence': 100,
              'datetime': datetime.datetime(2021, 5, 10, 4, 14, 10),
              'surface_temperature_celcius': 103}}
{'air_temperature_celcius': 10,
 'hotspots': {'confidence': 87,
              'datetime': datetime.datetime(2021, 5, 10

{'air_temperature_celcius': 10,
 'hotspots': {'confidence': 96,
              'datetime': datetime.datetime(2021, 5, 15, 4, 34, 30),
              'surface_temperature_celcius': 78}}
{'air_temperature_celcius': 10,
 'hotspots': {'confidence': 83,
              'datetime': datetime.datetime(2021, 5, 15, 4, 33),
              'surface_temperature_celcius': 57}}
{'air_temperature_celcius': 10,
 'hotspots': {'confidence': 92,
              'datetime': datetime.datetime(2021, 5, 15, 4, 33),
              'surface_temperature_celcius': 70}}
{'air_temperature_celcius': 10,
 'hotspots': {'confidence': 100,
              'datetime': datetime.datetime(2021, 5, 15, 4, 32, 50),
              'surface_temperature_celcius': 101}}
{'air_temperature_celcius': 10,
 'hotspots': {'confidence': 84,
              'datetime': datetime.datetime(2021, 5, 15, 4, 32, 50),
              'surface_temperature_celcius': 58}}
{'air_temperature_celcius': 10,
 'hotspots': {'confidence': 96,
              'datetime': d

{'air_temperature_celcius': 11,
 'hotspots': {'confidence': 81,
              'datetime': datetime.datetime(2021, 9, 10, 3, 50, 10),
              'surface_temperature_celcius': 54}}
{'air_temperature_celcius': 14,
 'hotspots': {'confidence': 89,
              'datetime': datetime.datetime(2021, 9, 20, 4, 29, 3),
              'surface_temperature_celcius': 60}}
{'air_temperature_celcius': 14,
 'hotspots': {'confidence': 100,
              'datetime': datetime.datetime(2021, 9, 20, 4, 29, 3),
              'surface_temperature_celcius': 112}}
{'air_temperature_celcius': 14,
 'hotspots': {'confidence': 88,
              'datetime': datetime.datetime(2021, 9, 20, 4, 28, 45),
              'surface_temperature_celcius': 60}}
{'air_temperature_celcius': 17,
 'hotspots': {'confidence': 86,
              'datetime': datetime.datetime(2021, 9, 23, 23, 50, 44),
              'surface_temperature_celcius': 56}}
{'air_temperature_celcius': 17,
 'hotspots': {'confidence': 100,
              'date

*e. Find the top 10 records with the highest surface temperature (°C).*

In [11]:
res = collection.aggregate([{'$unwind': '$hotspots'}, 
                            {'$sort': {'hotspots.surface_temperature_celcius': -1}},
                            {'$limit': 10}])

for data in res:
    pprint(data)

{'GHI_w/m2': 122,
 '_id': ObjectId('62894501763877b9f2d0e15d'),
 'air_temperature_celcius': 15,
 'date': datetime.datetime(2021, 4, 18, 0, 0),
 'hotspots': {'confidence': 100,
              'date': datetime.datetime(2021, 4, 18, 0, 0),
              'datetime': datetime.datetime(2021, 4, 18, 4, 52),
              'latitude': -38.1665,
              'longitude': 143.062,
              'surface_temperature_celcius': 124},
 'max_wind_speed': 9.9,
 'precipitation': 0.0,
 'precipitation_flag': 'I',
 'relative_humidity': 56.1,
 'station': 948701,
 'windspeed_knots': 5.1}
{'GHI_w/m2': 176,
 '_id': ObjectId('62894501763877b9f2d0e13e'),
 'air_temperature_celcius': 21,
 'date': datetime.datetime(2021, 3, 18, 0, 0),
 'hotspots': {'confidence': 100,
              'date': datetime.datetime(2021, 3, 18, 0, 0),
              'datetime': datetime.datetime(2021, 3, 18, 3, 50, 50),
              'latitude': -37.017,
              'longitude': 148.1297,
              'surface_temperature_celcius': 121},


*f. Find the number of fires each day. You are required to only display the total
number of fires and the date in the output.*

In [12]:
res = collection.find({}, {'_id': 0, 'number_of_fires': {'$size': '$hotspots'}, 'date': 1})

for data in res:
    pprint(data)

{'date': datetime.datetime(2020, 12, 31, 0, 0), 'number_of_fires': 0}
{'date': datetime.datetime(2021, 1, 2, 0, 0), 'number_of_fires': 0}
{'date': datetime.datetime(2021, 1, 3, 0, 0), 'number_of_fires': 0}
{'date': datetime.datetime(2021, 1, 4, 0, 0), 'number_of_fires': 0}
{'date': datetime.datetime(2021, 1, 5, 0, 0), 'number_of_fires': 0}
{'date': datetime.datetime(2021, 1, 6, 0, 0), 'number_of_fires': 0}
{'date': datetime.datetime(2021, 1, 7, 0, 0), 'number_of_fires': 0}
{'date': datetime.datetime(2021, 1, 8, 0, 0), 'number_of_fires': 0}
{'date': datetime.datetime(2021, 1, 9, 0, 0), 'number_of_fires': 0}
{'date': datetime.datetime(2021, 1, 10, 0, 0), 'number_of_fires': 0}
{'date': datetime.datetime(2021, 1, 11, 0, 0), 'number_of_fires': 0}
{'date': datetime.datetime(2021, 1, 12, 0, 0), 'number_of_fires': 0}
{'date': datetime.datetime(2021, 1, 13, 0, 0), 'number_of_fires': 0}
{'date': datetime.datetime(2021, 1, 14, 0, 0), 'number_of_fires': 0}
{'date': datetime.datetime(2021, 1, 15, 0

{'date': datetime.datetime(2021, 9, 12, 0, 0), 'number_of_fires': 0}
{'date': datetime.datetime(2021, 9, 13, 0, 0), 'number_of_fires': 0}
{'date': datetime.datetime(2021, 9, 14, 0, 0), 'number_of_fires': 0}
{'date': datetime.datetime(2021, 9, 15, 0, 0), 'number_of_fires': 0}
{'date': datetime.datetime(2021, 9, 16, 0, 0), 'number_of_fires': 0}
{'date': datetime.datetime(2021, 9, 17, 0, 0), 'number_of_fires': 0}
{'date': datetime.datetime(2021, 9, 18, 0, 0), 'number_of_fires': 0}
{'date': datetime.datetime(2021, 9, 19, 0, 0), 'number_of_fires': 0}
{'date': datetime.datetime(2021, 9, 20, 0, 0), 'number_of_fires': 5}
{'date': datetime.datetime(2021, 9, 21, 0, 0), 'number_of_fires': 2}
{'date': datetime.datetime(2021, 9, 22, 0, 0), 'number_of_fires': 1}
{'date': datetime.datetime(2021, 9, 23, 0, 0), 'number_of_fires': 23}
{'date': datetime.datetime(2021, 9, 24, 0, 0), 'number_of_fires': 28}
{'date': datetime.datetime(2021, 9, 25, 0, 0), 'number_of_fires': 0}
{'date': datetime.datetime(2021,

*g. Find the records of fires where the confidence is below 70.*

In [13]:
res = collection.aggregate([{'$unwind': '$hotspots'}, 
                            {'$match': {'hotspots.confidence': {'$lt': 70}}},
                            {'$project': {'_id': 0, 'hotspots': 1}}])

for data in res:
    pprint(data)

{'hotspots': {'confidence': 55,
              'date': datetime.datetime(2021, 3, 10, 0, 0),
              'datetime': datetime.datetime(2021, 3, 10, 4, 43),
              'latitude': -36.2544,
              'longitude': 148.0353,
              'surface_temperature_celcius': 42}}
{'hotspots': {'confidence': 54,
              'date': datetime.datetime(2021, 3, 10, 0, 0),
              'datetime': datetime.datetime(2021, 3, 10, 4, 42, 30),
              'latitude': -37.2197,
              'longitude': 147.9621,
              'surface_temperature_celcius': 43}}
{'hotspots': {'confidence': 56,
              'date': datetime.datetime(2021, 3, 13, 0, 0),
              'datetime': datetime.datetime(2021, 3, 13, 23, 58, 50),
              'latitude': -37.0286,
              'longitude': 148.155,
              'surface_temperature_celcius': 42}}
{'hotspots': {'confidence': 52,
              'date': datetime.datetime(2021, 3, 13, 0, 0),
              'datetime': datetime.datetime(2021, 3, 13, 12,

{'hotspots': {'confidence': 57,
              'date': datetime.datetime(2021, 4, 13, 0, 0),
              'datetime': datetime.datetime(2021, 4, 13, 4, 28, 30),
              'latitude': -38.0643,
              'longitude': 143.6616,
              'surface_temperature_celcius': 43}}
{'hotspots': {'confidence': 67,
              'date': datetime.datetime(2021, 4, 13, 0, 0),
              'datetime': datetime.datetime(2021, 4, 13, 4, 28, 30),
              'latitude': -37.9898,
              'longitude': 143.8507,
              'surface_temperature_celcius': 43}}
{'hotspots': {'confidence': 69,
              'date': datetime.datetime(2021, 4, 13, 0, 0),
              'datetime': datetime.datetime(2021, 4, 13, 4, 28, 10),
              'latitude': -37.0667,
              'longitude': 143.0582,
              'surface_temperature_celcius': 44}}
{'hotspots': {'confidence': 60,
              'date': datetime.datetime(2021, 4, 13, 0, 0),
              'datetime': datetime.datetime(2021, 4, 13,

              'surface_temperature_celcius': 43}}
{'hotspots': {'confidence': 64,
              'date': datetime.datetime(2021, 4, 13, 0, 0),
              'datetime': datetime.datetime(2021, 4, 13, 0, 14, 50),
              'latitude': -35.2465,
              'longitude': 143.4426,
              'surface_temperature_celcius': 42}}
{'hotspots': {'confidence': 58,
              'date': datetime.datetime(2021, 4, 13, 0, 0),
              'datetime': datetime.datetime(2021, 4, 13, 0, 8, 50),
              'latitude': -35.2432,
              'longitude': 143.4496,
              'surface_temperature_celcius': 44}}
{'hotspots': {'confidence': 65,
              'date': datetime.datetime(2021, 4, 13, 0, 0),
              'datetime': datetime.datetime(2021, 4, 13, 0, 7),
              'latitude': -34.5396,
              'longitude': 141.7292,
              'surface_temperature_celcius': 43}}
{'hotspots': {'confidence': 63,
              'date': datetime.datetime(2021, 4, 13, 0, 0),
            

              'surface_temperature_celcius': 41}}
{'hotspots': {'confidence': 66,
              'date': datetime.datetime(2021, 4, 18, 0, 0),
              'datetime': datetime.datetime(2021, 4, 18, 4, 44, 50),
              'latitude': -37.9388,
              'longitude': 143.7329,
              'surface_temperature_celcius': 43}}
{'hotspots': {'confidence': 59,
              'date': datetime.datetime(2021, 4, 18, 0, 0),
              'datetime': datetime.datetime(2021, 4, 18, 4, 44, 50),
              'latitude': -37.9718,
              'longitude': 143.2609,
              'surface_temperature_celcius': 40}}
{'hotspots': {'confidence': 63,
              'date': datetime.datetime(2021, 4, 18, 0, 0),
              'datetime': datetime.datetime(2021, 4, 18, 4, 44, 50),
              'latitude': -37.9176,
              'longitude': 143.43,
              'surface_temperature_celcius': 41}}
{'hotspots': {'confidence': 69,
              'date': datetime.datetime(2021, 4, 18, 0, 0),
        

{'hotspots': {'confidence': 58,
              'date': datetime.datetime(2021, 4, 18, 0, 0),
              'datetime': datetime.datetime(2021, 4, 18, 4, 44, 50),
              'latitude': -36.9203,
              'longitude': 143.4581,
              'surface_temperature_celcius': 40}}
{'hotspots': {'confidence': 64,
              'date': datetime.datetime(2021, 4, 18, 0, 0),
              'datetime': datetime.datetime(2021, 4, 18, 4, 44, 50),
              'latitude': -36.7898,
              'longitude': 144.008,
              'surface_temperature_celcius': 46}}
{'hotspots': {'confidence': 67,
              'date': datetime.datetime(2021, 4, 18, 0, 0),
              'datetime': datetime.datetime(2021, 4, 18, 4, 44, 50),
              'latitude': -36.9205,
              'longitude': 143.0889,
              'surface_temperature_celcius': 43}}
{'hotspots': {'confidence': 65,
              'date': datetime.datetime(2021, 4, 18, 0, 0),
              'datetime': datetime.datetime(2021, 4, 18, 

{'hotspots': {'confidence': 57,
              'date': datetime.datetime(2021, 4, 19, 0, 0),
              'datetime': datetime.datetime(2021, 4, 19, 3, 51, 10),
              'latitude': -35.9326,
              'longitude': 143.9772,
              'surface_temperature_celcius': 42}}
{'hotspots': {'confidence': 60,
              'date': datetime.datetime(2021, 4, 19, 0, 0),
              'datetime': datetime.datetime(2021, 4, 19, 3, 50, 50),
              'latitude': -36.2724,
              'longitude': 146.4168,
              'surface_temperature_celcius': 44}}
{'hotspots': {'confidence': 64,
              'date': datetime.datetime(2021, 4, 19, 0, 0),
              'datetime': datetime.datetime(2021, 4, 19, 3, 50, 30),
              'latitude': -36.0047,
              'longitude': 146.4774,
              'surface_temperature_celcius': 42}}
{'hotspots': {'confidence': 58,
              'date': datetime.datetime(2021, 4, 19, 0, 0),
              'datetime': datetime.datetime(2021, 4, 19,

              'datetime': datetime.datetime(2021, 5, 13, 4, 38, 40),
              'latitude': -34.9959,
              'longitude': 141.8635,
              'surface_temperature_celcius': 49}}
{'hotspots': {'confidence': 53,
              'date': datetime.datetime(2021, 5, 13, 0, 0),
              'datetime': datetime.datetime(2021, 5, 13, 4, 38, 40),
              'latitude': -34.9843,
              'longitude': 141.8736,
              'surface_temperature_celcius': 48}}
{'hotspots': {'confidence': 57,
              'date': datetime.datetime(2021, 5, 13, 0, 0),
              'datetime': datetime.datetime(2021, 5, 13, 4, 38, 40),
              'latitude': -36.8399,
              'longitude': 142.6291,
              'surface_temperature_celcius': 39}}
{'hotspots': {'confidence': 50,
              'date': datetime.datetime(2021, 5, 13, 0, 0),
              'datetime': datetime.datetime(2021, 5, 13, 4, 38, 40),
              'latitude': -35.708,
              'longitude': 143.7987,
       

{'hotspots': {'confidence': 66,
              'date': datetime.datetime(2021, 5, 15, 0, 0),
              'datetime': datetime.datetime(2021, 5, 15, 4, 26, 20),
              'latitude': -36.86,
              'longitude': 142.8137,
              'surface_temperature_celcius': 45}}
{'hotspots': {'confidence': 69,
              'date': datetime.datetime(2021, 5, 15, 0, 0),
              'datetime': datetime.datetime(2021, 5, 15, 4, 26, 20),
              'latitude': -36.3119,
              'longitude': 146.4371,
              'surface_temperature_celcius': 44}}
{'hotspots': {'confidence': 59,
              'date': datetime.datetime(2021, 5, 15, 0, 0),
              'datetime': datetime.datetime(2021, 5, 15, 4, 26, 20),
              'latitude': -36.6463,
              'longitude': 144.7594,
              'surface_temperature_celcius': 41}}
{'hotspots': {'confidence': 51,
              'date': datetime.datetime(2021, 5, 15, 0, 0),
              'datetime': datetime.datetime(2021, 5, 15, 4

              'date': datetime.datetime(2021, 11, 30, 0, 0),
              'datetime': datetime.datetime(2021, 11, 30, 0, 20, 53),
              'latitude': -37.6,
              'longitude': 149.297,
              'surface_temperature_celcius': 50}}
{'hotspots': {'confidence': 64,
              'date': datetime.datetime(2021, 11, 30, 0, 0),
              'datetime': datetime.datetime(2021, 11, 30, 0, 20, 49),
              'latitude': -37.382,
              'longitude': 149.341,
              'surface_temperature_celcius': 41}}
{'hotspots': {'confidence': 50,
              'date': datetime.datetime(2021, 12, 10, 0, 0),
              'datetime': datetime.datetime(2021, 12, 10, 0, 57, 59),
              'latitude': -37.987,
              'longitude': 144.005,
              'surface_temperature_celcius': 38}}
{'hotspots': {'confidence': 67,
              'date': datetime.datetime(2021, 12, 10, 0, 0),
              'datetime': datetime.datetime(2021, 12, 10, 0, 56, 57),
              'lati

*h. Find the average surface temperature (°C) for each day. You are required to
only display average surface temperature (°C) and the date in the output.*

In [14]:
res = collection.find({}, {'_id': 0, 'date': 1,
                           'avg_surface_temperature': {'$avg': '$hotspots.surface_temperature_celcius'}})

for data in res:
    pprint(data)

{'avg_surface_temperature': None, 'date': datetime.datetime(2020, 12, 31, 0, 0)}
{'avg_surface_temperature': None, 'date': datetime.datetime(2021, 1, 2, 0, 0)}
{'avg_surface_temperature': None, 'date': datetime.datetime(2021, 1, 3, 0, 0)}
{'avg_surface_temperature': None, 'date': datetime.datetime(2021, 1, 4, 0, 0)}
{'avg_surface_temperature': None, 'date': datetime.datetime(2021, 1, 5, 0, 0)}
{'avg_surface_temperature': None, 'date': datetime.datetime(2021, 1, 6, 0, 0)}
{'avg_surface_temperature': None, 'date': datetime.datetime(2021, 1, 7, 0, 0)}
{'avg_surface_temperature': None, 'date': datetime.datetime(2021, 1, 8, 0, 0)}
{'avg_surface_temperature': None, 'date': datetime.datetime(2021, 1, 9, 0, 0)}
{'avg_surface_temperature': None, 'date': datetime.datetime(2021, 1, 10, 0, 0)}
{'avg_surface_temperature': None, 'date': datetime.datetime(2021, 1, 11, 0, 0)}
{'avg_surface_temperature': None, 'date': datetime.datetime(2021, 1, 12, 0, 0)}
{'avg_surface_temperature': None, 'date': datet

*i. Find the top 10 records with the lowest GHI.*

In [15]:
res = collection.find().sort('GHI_w/m2', 1).limit(10)

for data in res:
    pprint(data)

{'GHI_w/m2': 47,
 '_id': ObjectId('62894501763877b9f2d0e1c7'),
 'air_temperature_celcius': 5,
 'date': datetime.datetime(2021, 8, 2, 0, 0),
 'hotspots': [],
 'max_wind_speed': 5.1,
 'precipitation': 0.0,
 'precipitation_flag': 'I',
 'relative_humidity': 38.6,
 'station': 948701,
 'windspeed_knots': 1.8}
{'GHI_w/m2': 48,
 '_id': ObjectId('62894501763877b9f2d0e1a6'),
 'air_temperature_celcius': 5,
 'date': datetime.datetime(2021, 6, 30, 0, 0),
 'hotspots': [{'confidence': 78,
               'date': datetime.datetime(2021, 6, 30, 0, 0),
               'datetime': datetime.datetime(2021, 6, 30, 4, 41, 25),
               'latitude': -36.834,
               'longitude': 142.524,
               'surface_temperature_celcius': 44},
              {'confidence': 78,
               'date': datetime.datetime(2021, 6, 30, 0, 0),
               'datetime': datetime.datetime(2021, 6, 30, 4, 41, 25),
               'latitude': -36.834,
               'longitude': 142.524,
               'surface_tempe

*j. Find the records with a 24-hour precipitation recorded between 0.20 to 0.35.*

In [16]:
res = collection.find({'precipitation_flag': {'$in': ['D', 'F', 'G']}, 
                       'precipitation': {'$gte': 0.20, '$lte': 0.35}})

for data in res:
    pprint(data)

{'GHI_w/m2': 157,
 '_id': ObjectId('62894501763877b9f2d0e0fe'),
 'air_temperature_celcius': 19,
 'date': datetime.datetime(2021, 1, 13, 0, 0),
 'hotspots': [],
 'max_wind_speed': 18.1,
 'precipitation': 0.31,
 'precipitation_flag': 'G',
 'relative_humidity': 54.1,
 'station': 948700,
 'windspeed_knots': 11.2}
{'GHI_w/m2': 146,
 '_id': ObjectId('62894501763877b9f2d0e149'),
 'air_temperature_celcius': 17,
 'date': datetime.datetime(2021, 3, 29, 0, 0),
 'hotspots': [{'confidence': 69,
               'date': datetime.datetime(2021, 3, 29, 0, 0),
               'datetime': datetime.datetime(2021, 3, 29, 0, 48, 40),
               'latitude': -34.2648,
               'longitude': 141.6325,
               'surface_temperature_celcius': 51}],
 'max_wind_speed': 21.0,
 'precipitation': 0.24,
 'precipitation_flag': 'G',
 'relative_humidity': 49.9,
 'station': 948701,
 'windspeed_knots': 12.2}
{'GHI_w/m2': 166,
 '_id': ObjectId('62894501763877b9f2d0e15f'),
 'air_temperature_celcius': 20,
 'date':

 'air_temperature_celcius': 11,
 'date': datetime.datetime(2021, 7, 3, 0, 0),
 'hotspots': [],
 'max_wind_speed': 18.1,
 'precipitation': 0.2,
 'precipitation_flag': 'G',
 'relative_humidity': 45.1,
 'station': 948701,
 'windspeed_knots': 11.5}
{'GHI_w/m2': 99,
 '_id': ObjectId('62894501763877b9f2d0e1c1'),
 'air_temperature_celcius': 11,
 'date': datetime.datetime(2021, 7, 27, 0, 0),
 'hotspots': [],
 'max_wind_speed': 21.0,
 'precipitation': 0.24,
 'precipitation_flag': 'G',
 'relative_humidity': 43.5,
 'station': 948701,
 'windspeed_knots': 13.1}
{'GHI_w/m2': 83,
 '_id': ObjectId('62894501763877b9f2d0e1cb'),
 'air_temperature_celcius': 9,
 'date': datetime.datetime(2021, 8, 6, 0, 0),
 'hotspots': [],
 'max_wind_speed': 19.0,
 'precipitation': 0.24,
 'precipitation_flag': 'G',
 'relative_humidity': 41.1,
 'station': 948702,
 'windspeed_knots': 12.7}
{'GHI_w/m2': 102,
 '_id': ObjectId('62894501763877b9f2d0e1df'),
 'air_temperature_celcius': 11,
 'date': datetime.datetime(2021, 8, 26, 0